In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import datetime
from matplotlib import pyplot as plt
import numpy as np

In [2]:
DEBUG = False

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
train_data = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False, num_workers=2)

classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
alexnet = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

alexnet.classifier[4] = nn.Linear(4096,1024)
alexnet.classifier[6] = nn.Linear(1024,10)

alexnet.load_state_dict(torch.load('model_20240525_150139_final'))
alexnet.eval()

Using cache found in C:\Users\Elijah/.cache\torch\hub\pytorch_vision_v0.6.0
C:\Users\Elijah\anaconda3\envs\cs190-project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Elijah\anaconda3\envs\cs190-project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
# # input : tensor output: binary string
# def binary(num):
#     return ''.join('{:0>8b}'.format(c) for c in struct.pack('!f', num))

# #input: mantissa bitstring
# #output: float value
# def calc_mantissa(mantissa):
#     res = 0
#     for k in range(len(mantissa)):
#         if mantissa[k] == '1':
#           res += 2**(-k-1)
#     return res

# #input exp: bitstring
# # new_exp_len: new length of exp
# def calc_exp(exp, new_exp_len):
#     limit = 2**(new_exp_len) - 1
#     # if exp is more than new_exp_len limit, truncate to new_exp_len limit.
#     bias = 2**(len(exp)-1) - 1
#     val = int(exp,2) - bias
#     if val > limit:
#         return limit
#     if val < -limit + 1:
#         return -limit + 1

#     return val

# def round_fp8(x, exp = 4):
#   '''
#   Quantizes input tensor to FP8 data format
#   inputs  x:      original tensor
#           exp:    number of bits used for exponent field
#                   e.g. E5M2 has 5 exp bits, E4M3 has 4 exp bits
#   output  x_32:   quantized tensor
#   '''

#   x_fp8 = x.clone().to(torch.float32)

#   for i in range(len(x_fp8)):
#     for j in range(len(x_fp8[i])):
#       result = 1.0
#       bin_str = binary(x[i][j])

#       bin_mantissa = bin_str[9:32]
#       res_mantissa = bin_mantissa[:7-exp]    
#       result += calc_mantissa(res_mantissa)

#       bin_exp = bin_str[1:9]
#       exp_int = calc_exp(bin_exp, exp)
#       result *= 2**exp_int

#       if bin_str[0] == '1':
#         result *= -1

#       x_fp8[i][j] = result    

#   return x_fp8.to(torch.float32)

# def dt_dequantize(quantized):
#     """
#     Custom dequantization function.
#     Assumes quantized values are in the custom format.
#     """
#     dequantized = torch.zeros_like(quantized, dtype=torch.int32)

#     for i, qval in enumerate(quantized.view(-1)):
#         # Extract the bits
#         sign_bit = (qval >> 7) & 1
#         exp_bits = (qval >> 3) & 0xF
#         bis_flag = (qval >> 2) & 1
#         bis_tree = qval & 0x3

#         # Compute the base value from the bisection tree bits
#         base_value = bis_tree / 4.0

#         # If bisection flag is set, adjust the base value
#         if bis_flag:
#             base_value += 0.5 / 4.0

#         # Compute the dequantized value using the exponent
#         value = base_value * (10 ** -exp_bits)


# #the input is normalized tensor x,
# def round_dt8(x, exp = 4):
#     x_dt8 = x.clone().to(torch.float32)
#     # print(f'DT quantizaiton; cloned shape: {x_dt8.shape}')
#     for i, row in enumerate(x_dt8):
#         # print(f'Iterating over rows: {x_dt8.shape}')
#         for j, val in enumerate(row):
#             # Determine sign bit
#             sign_bit = 0 if val >= 0 else 1

#             # Absolute value for further processing
#             abs_val = abs(val)
#                     # Determine the exponent bits (4 bits)
#             exp_bits = 0
#             for k in range(15):
#                 if abs_val < 10**-k:
#                     exp_bits = k - 1
#                     break

#             # Determine the bisection tree flag and binary bisection tree bits (3 bits)
#             bis_flag = 1 if abs_val % (10**-exp_bits) != 0 else 0
#             bis_tree = int((abs_val / (10**-exp_bits)) * 4) % 4

#             # Combine the bits
#             row[j] = (sign_bit << 7) | (exp_bits << 3) | (bis_flag << 2) | bis_tree
#             # row[j] = dt_dequantize(row[j])

#     return x_dt8

# def quantize_rowwise(x: torch.Tensor, dt = False):
#     abso = torch.abs(x)
#     output_maxs  = torch.max(abso,1)[0].unsqueeze(-1)
#     output = x[0]  / output_maxs[0,None] # What is this doing? Why x[0]?
#     if not dt:
#         output = round_fp8(output)
#     else:
#         output = round_dt8(output)
#     return output, output_maxs

# def dequantize_rowwise(x: torch.Tensor, state_x: torch.Tensor):
#     output = x * state_x
#     return output

# def init_weights(m):
#     if isinstance(m, nn.Linear):
#         init.normal_(m.weight, mean=0.0, std=1.0)
#         if m.bias is not None:
#             init.zeros_(m.bias)

In [10]:
import torch
import torch.nn as nn
import math
import copy
import random
import numpy as np
import struct
import torch.nn.init as init

# input : tensor output: binary string
def binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!f', num))

#input: mantissa bitstring
#output: float value
def calc_mantissa(mantissa):
    res = 0
    for k in range(len(mantissa)):
        if mantissa[k] == '1':
          res += 2**(-k-1)
    return res

#input exp: bitstring
# new_exp_len: new length of exp
def calc_exp(exp, new_exp_len):
    limit = 2**(new_exp_len) - 1
    # if exp is more than new_exp_len limit, truncate to new_exp_len limit.
    bias = 2**(len(exp)-1) - 1
    val = int(exp,2) - bias
    if val > limit:
        return limit
    if val < -limit + 1:
        return -limit + 1

    return val

def round_fp8(x, exp = 4):
  '''
  Quantizes input tensor to FP8 data format
  inputs  x:      original tensor
          exp:    number of bits used for exponent field
                  e.g. E5M2 has 5 exp bits, E4M3 has 4 exp bits
  output  x_32:   quantized tensor
  '''

  x_fp8 = x.clone().to(torch.float32)

  for i in range(len(x_fp8)):
    for j in range(len(x_fp8[i])):
      result = 1.0
      bin_str = binary(x[i][j])

      bin_mantissa = bin_str[9:32]
      res_mantissa = bin_mantissa[:7-exp]    
      result += calc_mantissa(res_mantissa)

      bin_exp = bin_str[1:9]
      exp_int = calc_exp(bin_exp, exp)
      result *= 2**exp_int

      if bin_str[0] == '1':
        result *= -1

      x_fp8[i][j] = result    

  return x_fp8.to(torch.float32)
def bisection_quantization(num, bits = 7):
    val = abs(num)
    inversed_bits = []
    # Bisection tree quantization
    range_min, range_max = 0, 1
    for _ in range(bits):
        mid = (range_min + range_max) / 2
        if val >= mid:
            inversed_bits.append(1)
            range_min = mid
        else:
            inversed_bits.append(0)
            range_max = mid

    quantized_val = 0
    for k, bit in enumerate(inversed_bits):
        if bit:
            quantized_val += 2**-(k + 1)
            
    return quantized_val
    
#the input is normalized tensor x,
def round_dt8(x, exp = 4):
    x_dt8 = x.clone().to(torch.float32)
    output = torch.zeros_like(x_dt8)
    num_levels = 2 ** (7)

    for i, row in enumerate(x_dt8):
        for j, val in enumerate(row):
            sign_bit = 0 if val >= 0 else 1
            val = abs(val)
            exp_bits = 0
            while val < 0.1:
                val *= 10
                exp_bits += 1

            bs_bits = max(0, 6 - exp_bits)
            exp_bits = min(7, exp_bits)

            if exp_bits == 0:
                quantized_val = bisection_quantization(val, 7)
            elif exp_bits >= 6:
                quantized_val = val
            else:
                quantized_val = bisection_quantization(val, bs_bits)
            # print(quantized_val,x[i,j])
            quantized_val = quantized_val if sign_bit == 0 else -quantized_val
            # print(np.sign(quantized_val), np.sign(x[i,j]))
            # assert(np.sign(quantized_val) == np.sign(x[i,j]))
            quantized_val *= 10**(-exp_bits)
            output[i, j] = quantized_val

    return output

def quantize_rowwise(x: torch.Tensor, dt = True):
    abso = torch.abs(x)
    output_maxs  = torch.max(abso,1)[0].unsqueeze(-1)
    output = x[0]  / output_maxs[0,None]
    if not dt:
        output = round_fp8(output)
    else:
        output = round_dt8(output)
    return output, output_maxs

def dequantize_rowwise(x: torch.Tensor, state_x: torch.Tensor):
    output = x * state_x
    return output

def init_weights(m):
    if isinstance(m, nn.Linear):
        init.normal_(m.weight, mean=0.0, std=1.0)
        if m.bias is not None:
            init.zeros_(m.bias)

In [8]:
def quantize_dequantize_dt(mat):
    # return quantize_rowwise(mat, dt = True)[0]
    testing_dt, dt_max = quantize_rowwise(mat, dt = True)
    return dequantize_rowwise(testing_dt,dt_max)

In [9]:
if not DEBUG:
    count = 0
    for layer in [*alexnet.features,*alexnet.classifier]:
        count += 1
        try:
            if len(layer.weight.shape) == 4:
                weights = layer.weight.detach()
                print(f'Layer {count} weights shape pre-quantization: {weights.shape}\nWeights: {weights}')
                for filter in range(0, layer.weight.shape[0]):
                    for channel in range(0, layer.weight.shape[1]):
                        # print(layer.weight[filter,channel])
                        weights[filter,channel] = quantize_dequantize_dt(weights[filter,channel])
                print(f'Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
                layer.weight = nn.parameter.Parameter(weights)
            else:
                print(f'Layer {count} weights shape pre-quantization: {layer.weight.shape}\nWeights: {weights}')
                layer.weight = nn.parameter.Parameter(quantize_dequantize_dt(layer.weight.detach()))
                print(f'Layer {count} weights shape post-quantization: {layer.weight.shape}\nWeights: {weights}')
                # print(layer.weight)
        except (TypeError, AttributeError):
            pass
else:
    count = 0
    for layer in alexnet.classifier:
        count += 1
        try:
            if len(layer.weight.shape) == 4:
                weights = layer.weight.detach()
                print(f'Layer {count} weights shape pre-quantization: {weights.shape}')
                
                for filter in range(0, layer.weight.shape[0]):
                    for channel in range(0, layer.weight.shape[1]):
                        # print(layer.weight[filter,channel])
                        weights[filter,channel] = quantize_dequantize_dt(weights[filter,channel])
                print(f'Layer {count} weights shape post-quantization: {weights.shape}')
                layer.weight = nn.parameter.Parameter(weights)
            else:
                print(f'In else loop')
                print(f'Layer {count} weights shape pre-quantization: {layer.weight.shape}')
                intermediate = quantize_dequantize_dt(layer.weight.detach())
                print(intermediate.shape)
                layer.weight = nn.parameter.Parameter(intermediate)
                print(f'Layer {count} weights shape post-quantization: {layer.weight.shape}')
                # print(layer.weight)
        except (TypeError, AttributeError):
            pass

# for layer in [*alexnet.features,*alexnet.classifier]:
#     try:
#         if len(layer.weight.shape) == 4:
#         for filter in range(0, layer.weight.shape[0]):
#             for channel in 
#         layer.weight = nn.parameter.Parameter(quantize_dequantize_dt(layer.weight.detach()))
#     except (TypeError, AttributeError):
#         print(layer)

# for layer in [*alexnet.features,*alexnet.classifier]:
#     try:
#         if len(layer.weight.shape) == 4:
#         for filter in range(0, layer.weight.shape[0]):
#             for channel in 
#         layer.weight = nn.parameter.Parameter(quantize_dequantize_dt(layer.weight.detach()))
#     except (TypeError, AttributeError):
#         print(layer)


Layer 1 weights shape pre-quantization: torch.Size([64, 3, 11, 11])
Weights: tensor([[[[ 1.9734e-01,  1.5311e-01,  1.2346e-01,  ...,  5.1259e-02,
            1.8628e-02,  4.9283e-02],
          [ 1.3495e-01,  8.2730e-02,  6.8725e-02,  ...,  2.2493e-02,
           -1.1554e-02,  7.1114e-03],
          [ 1.0532e-01,  5.7741e-02,  4.7859e-02,  ...,  3.1955e-02,
            4.0302e-03,  9.6770e-03],
          ...,
          [ 8.5962e-02,  1.0595e-01,  5.6914e-02,  ..., -2.0370e-01,
           -1.3139e-01, -1.2237e-01],
          [ 3.4958e-02,  6.8188e-02,  2.8633e-02,  ..., -2.0155e-01,
           -1.1676e-01, -1.2000e-01],
          [ 3.9003e-02,  6.7265e-02,  2.0207e-02,  ..., -1.2560e-01,
           -1.0937e-01, -1.0958e-01]],

         [[-8.0962e-02, -7.6534e-02, -1.2334e-01,  ...,  8.2895e-03,
           -1.7828e-03,  6.0680e-02],
          [-9.2570e-02, -9.6053e-02, -1.2600e-01,  ...,  1.5380e-02,
            5.2623e-03,  5.8764e-02],
          [-1.4768e-01, -1.3341e-01, -1.7059e-01, 

AssertionError: 

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
alexnet.to(device)

In [ ]:
#Testing Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = alexnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

In [ ]:
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

In [ ]:
model_path = 'model_{}_{}'.format(timestamp, "DT_Quantized")
torch.save(alexnet.state_dict(), model_path)

---

In [52]:
print(f'Alexnet Layer: {alexnet.features[0]}\nShape: {alexnet.features[0].weight.shape}')

Alexnet Layer: Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
Shape: torch.Size([64, 3, 11, 11])


In [36]:
alexnet.features[0].weight[0][0][2]

tensor([0.1053, 0.0577, 0.0479, 0.0451, 0.0370, 0.0314, 0.0281, 0.0106, 0.0320,
        0.0040, 0.0097], grad_fn=<SelectBackward0>)

# DT Quantization Testing

In [120]:
import torch
import torch.nn as nn
import math
import copy
import random
import numpy as np
import struct
import torch.nn.init as init

# input : tensor output: binary string
def binary(num):
    return ''.join('{:0>8b}'.format(c) for c in struct.pack('!f', num))

#input: mantissa bitstring
#output: float value
def calc_mantissa(mantissa):
    res = 0
    for k in range(len(mantissa)):
        if mantissa[k] == '1':
          res += 2**(-k-1)
    return res

#input exp: bitstring
# new_exp_len: new length of exp
def calc_exp(exp, new_exp_len):
    limit = 2**(new_exp_len) - 1
    # if exp is more than new_exp_len limit, truncate to new_exp_len limit.
    bias = 2**(len(exp)-1) - 1
    val = int(exp,2) - bias
    if val > limit:
        return limit
    if val < -limit + 1:
        return -limit + 1

    return val

def round_fp8(x, exp = 4):
  '''
  Quantizes input tensor to FP8 data format
  inputs  x:      original tensor
          exp:    number of bits used for exponent field
                  e.g. E5M2 has 5 exp bits, E4M3 has 4 exp bits
  output  x_32:   quantized tensor
  '''

  x_fp8 = x.clone().to(torch.float32)

  for i in range(len(x_fp8)):
    for j in range(len(x_fp8[i])):
      result = 1.0
      bin_str = binary(x[i][j])

      bin_mantissa = bin_str[9:32]
      res_mantissa = bin_mantissa[:7-exp]    
      result += calc_mantissa(res_mantissa)

      bin_exp = bin_str[1:9]
      exp_int = calc_exp(bin_exp, exp)
      result *= 2**exp_int

      if bin_str[0] == '1':
        result *= -1

      x_fp8[i][j] = result    

  return x_fp8.to(torch.float32)
def bisection_quantization(num, bits = 7):
    val = abs(num)
    inversed_bits = []
    # Bisection tree quantization
    range_min, range_max = 0, 1
    for _ in range(bits):
        mid = (range_min + range_max) / 2
        if val >= mid:
            inversed_bits.append(1)
            range_min = mid
        else:
            inversed_bits.append(0)
            range_max = mid

    quantized_val = 0
    for k, bit in enumerate(inversed_bits):
        if bit:
            quantized_val += 2**-(k + 1)
            
    return quantized_val
    
#the input is normalized tensor x,
def round_dt8(x, exp = 4):
    x_dt8 = x.clone().to(torch.float32)
    output = torch.zeros_like(x_dt8)
    num_levels = 2 ** (7)

    for i, row in enumerate(x_dt8):
        for j, val in enumerate(row):
            sign_bit = 0 if val >= 0 else 1
            val = abs(val)
            exp_bits = 0
            while val < 0.1:
                val *= 10
                exp_bits += 1

            bs_bits = max(0, 6 - exp_bits)
            exp_bits = min(7, exp_bits)

            if exp_bits == 0:
                quantized_val = bisection_quantization(val, 7)
            elif exp_bits >= 6:
                quantized_val = val
            else:
                quantized_val = bisection_quantization(val, bs_bits)
            # print(quantized_val,x[i,j])
            quantized_val = quantized_val if sign_bit == 0 else -quantized_val
            # print(np.sign(quantized_val), np.sign(x[i,j]))
            # assert(np.sign(quantized_val) == np.sign(x[i,j]))
            quantized_val *= 10**(-exp_bits)
            output[i, j] = quantized_val

    return output

def quantize_rowwise(x: torch.Tensor, dt = True):
    abso = torch.abs(x)
    output_maxs  = torch.max(abso,1)[0].unsqueeze(-1)
    output = x[0]  / output_maxs[0,None]
    if not dt:
        output = round_fp8(output)
    else:
        output = round_dt8(output)
    return output, output_maxs

def dequantize_rowwise(x: torch.Tensor, state_x: torch.Tensor):
    output = x * state_x
    return output

def init_weights(m):
    if isinstance(m, nn.Linear):
        init.normal_(m.weight, mean=0.0, std=1.0)
        if m.bias is not None:
            init.zeros_(m.bias)

In [121]:
# def quantize_dequantize_dt(mat):
#     # return quantize_rowwise(mat, dt = True)[0]
#     testing_dt, dt_max = quantize_rowwise(mat, dt = True)
#     return dequantize_rowwise(testing_dt,dt_max)

3 channels, 2x3 

In [39]:
conv1 = nn.Conv2d(3, 2, 3)
weight = conv1.weight.data.numpy()
print(weight[0,0])
print(f'Shape: {weight[0,0].shape}')

[[-0.0698105  -0.06304418 -0.1270199 ]
 [-0.14880249  0.14225613  0.06208219]
 [-0.12386654  0.14878742 -0.08242814]]
Shape: (3, 3)


In [40]:
# Before Quantization
conv1.weight

Parameter containing:
tensor([[[[-0.0698, -0.0630, -0.1270],
          [-0.1488,  0.1423,  0.0621],
          [-0.1239,  0.1488, -0.0824]],

         [[ 0.0952,  0.1482, -0.1008],
          [ 0.1148, -0.0182, -0.1181],
          [-0.0069, -0.1901,  0.1144]],

         [[ 0.0164,  0.0424, -0.1293],
          [-0.1826, -0.0776, -0.0721],
          [ 0.0227, -0.0045, -0.1900]]],


        [[[ 0.1061, -0.1882, -0.0699],
          [-0.0797, -0.0104,  0.1512],
          [ 0.1826,  0.0391,  0.0962]],

         [[-0.0984,  0.0195, -0.0787],
          [ 0.0441, -0.0458, -0.1090],
          [ 0.1836,  0.1033,  0.0996]],

         [[-0.1855,  0.0421, -0.1038],
          [ 0.0772, -0.1856, -0.0349],
          [-0.1116,  0.1852, -0.0794]]]], requires_grad=True)

In [41]:
weights = conv1.weight.detach()
old_weight = weights.clone()
print(f'Layer {count} weights shape pre-quantization: {weights.shape}\nWeights: {weights}')
for filter in range(0, weights.shape[0]):
    # print(f'Filter num {filter}')
    for channel in range(0, weights.shape[1]):
        # print(f'Channel num {channel}')
        # print(layer.weight[filter,channel])
        weights[filter,channel] = quantize_dequantize_dt(weights[filter,channel])
        # print(f'Finish window')
print(f'Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
# layer.weight = nn.parameter.Parameter(weights)

Layer 20 weights shape pre-quantization: torch.Size([2, 3, 3, 3])
Weights: tensor([[[[-0.0698, -0.0630, -0.1270],
          [-0.1488,  0.1423,  0.0621],
          [-0.1239,  0.1488, -0.0824]],

         [[ 0.0952,  0.1482, -0.1008],
          [ 0.1148, -0.0182, -0.1181],
          [-0.0069, -0.1901,  0.1144]],

         [[ 0.0164,  0.0424, -0.1293],
          [-0.1826, -0.0776, -0.0721],
          [ 0.0227, -0.0045, -0.1900]]],


        [[[ 0.1061, -0.1882, -0.0699],
          [-0.0797, -0.0104,  0.1512],
          [ 0.1826,  0.0391,  0.0962]],

         [[-0.0984,  0.0195, -0.0787],
          [ 0.0441, -0.0458, -0.1090],
          [ 0.1836,  0.1033,  0.0996]],

         [[-0.1855,  0.0421, -0.1038],
          [ 0.0772, -0.1856, -0.0349],
          [-0.1116,  0.1852, -0.0794]]]])
0.546875 tensor(-0.5496)
-1.0 tensor(-1.)
0.4921875 tensor(-0.4963)
-1.0 tensor(-1.)
0.9921875 tensor(-1.)
-1.0 tensor(-1.)
0.640625 tensor(0.6423)
1.0 tensor(1.)
0.9921875 tensor(1.)
1.0 tensor(1.)
0.6796875

In [97]:
weights

tensor([[[[ 0.0669, -0.0191,  0.1103],
          [ 0.0951, -0.0272,  0.1569],
          [ 0.0797, -0.0228,  0.1314]],

         [[-0.0729, -0.1542,  0.1226],
          [-0.0438, -0.0926,  0.0737],
          [-0.0641, -0.1356,  0.1078]],

         [[-0.0153,  0.1499, -0.0814],
          [-0.0160,  0.1562, -0.0849],
          [-0.0129,  0.1262, -0.0686]]],


        [[[ 0.1128,  0.0408,  0.1003],
          [ 0.1421,  0.0515,  0.1265],
          [ 0.0809,  0.0293,  0.0720]],

         [[-0.0682, -0.0300, -0.1731],
          [-0.0426, -0.0187, -0.1081],
          [-0.0676, -0.0297, -0.1716]],

         [[-0.1728,  0.0204,  0.1252],
          [-0.0904,  0.0107,  0.0655],
          [-0.1685,  0.0199,  0.1221]]]])

In [43]:
torch.sign(weights) == torch.sign(old_weight)

tensor([[[[ True,  True,  True],
          [ True, False, False],
          [ True, False,  True]],

         [[ True,  True,  True],
          [ True, False,  True],
          [False, False, False]],

         [[ True,  True,  True],
          [False, False,  True],
          [ True, False,  True]]],


        [[[ True,  True,  True],
          [False,  True, False],
          [ True, False, False]],

         [[ True,  True,  True],
          [False, False,  True],
          [False,  True, False]],

         [[ True,  True,  True],
          [False, False,  True],
          [ True,  True,  True]]]])

In [98]:
# layer = conv1
# weights = 
# if len(layer.weight.shape) == 4:
#     for f in range(0, layer.weight.shape[0]):
#         for channel in range(0, layer.weight.shape[1]):
#             # print(layer.weight[filter,channel])
#             layer.weight[f,channel] = quantize_dequantize_dt(layer.weight.detach()[f,channel])

3 channels, 5x9

In [11]:
conv1 = nn.Conv2d(3, 5, 9)
weight = conv1.weight.data.numpy()
print(weight[0,0])
print(f'Shape: {weight[0,0].shape}')

[[ 5.5808768e-02 -1.8281944e-02  3.5957582e-02 -2.7421087e-02
  -6.4216554e-05 -1.7654877e-02 -4.6642348e-03 -4.4371106e-02
  -4.4541955e-02]
 [ 2.0261742e-02 -3.7637256e-02  6.3099809e-02 -3.7367452e-02
  -9.3750842e-03 -4.1278601e-02 -5.3225860e-02 -8.8727027e-03
  -4.9445525e-02]
 [-3.8906619e-02  1.0191157e-02  1.6317122e-02  1.1761315e-02
   2.4575636e-02  2.1294929e-02 -3.3873226e-02 -5.6855984e-02
   3.7869923e-02]
 [ 2.6974671e-02  6.0708523e-02  6.0964309e-02 -4.4884644e-02
  -1.0531750e-02  2.9396132e-02 -4.3708935e-02  3.4439944e-02
  -2.3533065e-02]
 [ 1.4285475e-02  6.4307749e-03  2.7122252e-02 -4.7616020e-02
   5.6815118e-02  6.1218224e-02  4.6352074e-03 -2.0432644e-02
  -2.4116155e-02]
 [ 5.1382810e-02  5.2728266e-02  5.2836470e-02 -4.2256638e-02
  -5.4871783e-02  3.2509796e-02  9.9337772e-03  2.7528040e-02
  -2.3554422e-02]
 [-1.5762970e-02 -5.3656846e-03  1.4798842e-02 -2.3912363e-02
   4.7903523e-02 -3.2052271e-02 -4.5687594e-02  7.5899065e-04
  -3.3366606e-03]
 [ 2.0

In [12]:
# Before Quantization
conv1.weight

Parameter containing:
tensor([[[[ 5.5809e-02, -1.8282e-02,  3.5958e-02,  ..., -4.6642e-03,
           -4.4371e-02, -4.4542e-02],
          [ 2.0262e-02, -3.7637e-02,  6.3100e-02,  ..., -5.3226e-02,
           -8.8727e-03, -4.9446e-02],
          [-3.8907e-02,  1.0191e-02,  1.6317e-02,  ..., -3.3873e-02,
           -5.6856e-02,  3.7870e-02],
          ...,
          [-1.5763e-02, -5.3657e-03,  1.4799e-02,  ..., -4.5688e-02,
            7.5899e-04, -3.3367e-03],
          [ 2.0130e-02, -3.1654e-02,  2.0753e-02,  ..., -1.3731e-02,
            2.7963e-02, -3.4471e-02],
          [-1.4438e-02, -2.5631e-02,  1.6019e-02,  ..., -6.0822e-02,
           -3.9687e-02, -8.1770e-03]],

         [[-4.7814e-02, -3.5626e-02, -2.7190e-03,  ...,  5.3565e-02,
            2.7545e-02,  4.5141e-02],
          [ 1.5875e-03,  5.6097e-03,  2.5978e-02,  ..., -4.3523e-02,
           -2.6759e-02, -2.1907e-02],
          [-1.7270e-03,  5.0312e-02, -5.6086e-02,  ..., -4.3808e-02,
           -4.5693e-02,  5.5960e-02]

In [13]:
weights = conv1.weight.detach()
print(f'Layer {count} weights shape pre-quantization: {weights.shape}\nWeights: {weights}')
for filter in range(0, weights.shape[0]):
    print(f'Filter num {filter}')
    for channel in range(0, weights.shape[1]):
        print(f'Channel num {channel}')
        # print(layer.weight[filter,channel])
        weights[filter,channel] = quantize_dequantize_dt(weights[filter,channel])
        print(f'Finish window')
print(f'Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
# layer.weight = nn.parameter.Parameter(weights)weights = conv1.weight.detach()
# print(f'Layer {count} weights shape pre-quantization: {weights.shape}\nWeights: {weights}')
# for filter in range(0, weights.shape[0]):
#     print(f'Filter num {filter}')
#     for channel in range(0, weights.shape[1]):
#         print(f'Channel num {channel}')
#         # print(layer.weight[filter,channel])
#         weights[filter,channel] = quantize_dequantize_dt(weights[filter,channel])
#         print(f'Finish window')
# print(f'Layer {count} weights shape post-quantization: {weights.shape}\nWeights: {weights}')
# # layer.weight = nn.parameter.Parameter(weights)

Layer 4 weights shape pre-quantization: torch.Size([5, 3, 9, 9])
Weights: tensor([[[[ 5.5809e-02, -1.8282e-02,  3.5958e-02,  ..., -4.6642e-03,
           -4.4371e-02, -4.4542e-02],
          [ 2.0262e-02, -3.7637e-02,  6.3100e-02,  ..., -5.3226e-02,
           -8.8727e-03, -4.9446e-02],
          [-3.8907e-02,  1.0191e-02,  1.6317e-02,  ..., -3.3873e-02,
           -5.6856e-02,  3.7870e-02],
          ...,
          [-1.5763e-02, -5.3657e-03,  1.4799e-02,  ..., -4.5688e-02,
            7.5899e-04, -3.3367e-03],
          [ 2.0130e-02, -3.1654e-02,  2.0753e-02,  ..., -1.3731e-02,
            2.7963e-02, -3.4471e-02],
          [-1.4438e-02, -2.5631e-02,  1.6019e-02,  ..., -6.0822e-02,
           -3.9687e-02, -8.1770e-03]],

         [[-4.7814e-02, -3.5626e-02, -2.7190e-03,  ...,  5.3565e-02,
            2.7545e-02,  4.5141e-02],
          [ 1.5875e-03,  5.6097e-03,  2.5978e-02,  ..., -4.3523e-02,
           -2.6759e-02, -2.1907e-02],
          [-1.7270e-03,  5.0312e-02, -5.6086e-02,  ..

In [14]:
weights

tensor([[[[ 0.9922, -0.3203,  0.6406,  ..., -0.0812, -0.7891, -0.7969],
          [ 0.9922, -0.3203,  0.6406,  ..., -0.0812, -0.7891, -0.7969],
          [ 0.9922, -0.3203,  0.6406,  ..., -0.0812, -0.7891, -0.7969],
          ...,
          [ 0.9922, -0.3203,  0.6406,  ..., -0.0812, -0.7891, -0.7969],
          [ 0.9922, -0.3203,  0.6406,  ..., -0.0812, -0.7891, -0.7969],
          [ 0.9922, -0.3203,  0.6406,  ..., -0.0812, -0.7891, -0.7969]],

         [[-0.8906, -0.6641, -0.0500,  ...,  0.9922,  0.5078,  0.8359],
          [-0.8906, -0.6641, -0.0500,  ...,  0.9922,  0.5078,  0.8359],
          [-0.8906, -0.6641, -0.0500,  ...,  0.9922,  0.5078,  0.8359],
          ...,
          [-0.8906, -0.6641, -0.0500,  ...,  0.9922,  0.5078,  0.8359],
          [-0.8906, -0.6641, -0.0500,  ...,  0.9922,  0.5078,  0.8359],
          [-0.8906, -0.6641, -0.0500,  ...,  0.9922,  0.5078,  0.8359]],

         [[ 0.0594,  0.4609,  0.6797,  ..., -0.9922,  0.0469,  0.2812],
          [ 0.0594,  0.4609,  

In [15]:
weights[0]

tensor([[[ 9.9219e-01, -3.2031e-01,  6.4062e-01, -4.8438e-01, -6.2500e-04,
          -3.1250e-01, -8.1250e-02, -7.8906e-01, -7.9688e-01],
         [ 9.9219e-01, -3.2031e-01,  6.4062e-01, -4.8438e-01, -6.2500e-04,
          -3.1250e-01, -8.1250e-02, -7.8906e-01, -7.9688e-01],
         [ 9.9219e-01, -3.2031e-01,  6.4062e-01, -4.8438e-01, -6.2500e-04,
          -3.1250e-01, -8.1250e-02, -7.8906e-01, -7.9688e-01],
         [ 9.9219e-01, -3.2031e-01,  6.4062e-01, -4.8438e-01, -6.2500e-04,
          -3.1250e-01, -8.1250e-02, -7.8906e-01, -7.9688e-01],
         [ 9.9219e-01, -3.2031e-01,  6.4062e-01, -4.8438e-01, -6.2500e-04,
          -3.1250e-01, -8.1250e-02, -7.8906e-01, -7.9688e-01],
         [ 9.9219e-01, -3.2031e-01,  6.4062e-01, -4.8438e-01, -6.2500e-04,
          -3.1250e-01, -8.1250e-02, -7.8906e-01, -7.9688e-01],
         [ 9.9219e-01, -3.2031e-01,  6.4062e-01, -4.8438e-01, -6.2500e-04,
          -3.1250e-01, -8.1250e-02, -7.8906e-01, -7.9688e-01],
         [ 9.9219e-01, -3.2031e-01

In [142]:
alexnet.features[0].weight[0]

tensor([[[ 0.0000, -0.7894, -0.7894, -0.7894, -0.7894, -0.7894, -0.7894,
          -0.7894, -0.7894, -0.7894, -0.7894],
         [ 0.0000, -0.5398, -0.5398, -0.5398, -0.5398, -0.5398, -0.5398,
          -0.5398, -0.5398, -0.5398, -0.5398],
         [ 0.0000, -0.4213, -0.4213, -0.4213, -0.4213, -0.4213, -0.4213,
          -0.4213, -0.4213, -0.4213, -0.4213],
         [ 0.0000, -0.3654, -0.3654, -0.3654, -0.3654, -0.3654, -0.3654,
          -0.3654, -0.3654, -0.3654, -0.3654],
         [ 0.0000, -0.3950, -0.3950, -0.3950, -0.3950, -0.3950, -0.3950,
          -0.3950, -0.3950, -0.3950, -0.3950],
         [ 0.0000, -0.4068, -0.4068, -0.4068, -0.4068, -0.4068, -0.4068,
          -0.4068, -0.4068, -0.4068, -0.4068],
         [ 0.0000, -0.5804, -0.5804, -0.5804, -0.5804, -0.5804, -0.5804,
          -0.5804, -0.5804, -0.5804, -0.5804],
         [ 0.0000, -0.6692, -0.6692, -0.6692, -0.6692, -0.6692, -0.6692,
          -0.6692, -0.6692, -0.6692, -0.6692],
         [ 0.0000, -0.8148, -0.8148, -0.

In [64]:
layer.weight.detach()[filter,channel]

tensor([[-0.0799,  0.0143,  0.0621],
        [-0.1410,  0.1376,  0.1556],
        [ 0.1617,  0.1489,  0.0450]])